In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

#불러오기,결측값제거
df = pd.read_csv('./list.csv',sep=',')
df = df.dropna()

#장학종류 더미변수화
df = df.join(df["장학종류"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#선발대상 더미변수화
df = df.join(df["선발대상"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#조회수
df["조회 수"]=df["조회수"].str.extract(r'(\d+)')    #'조회:'없애기
df=df.astype({"조회 수":'int'})

#선발인원
df["선발 인원(명)"]=df["선발인원"].str.extract(r'(\d+)')    #숫자만 남기기
df=df.replace({"선발 인원(명)":'000'},'999')                #000은 세자리수를 의미함으로 999명으로 변경
df=df.replace({"선발 인원(명)":'00'},'99')                  #00은 두자리수를 의미함으로 99명으로 변경
df=df.replace({"선발 인원(명)":'0'},'9')                    #0은 힌자리수를 의미함으로 9명으로 변경
df=df.replace({"선발 인원(명)":np.NaN},0)                   #NaN(미정)값 0으로 변경
df=df.astype({"선발 인원(명)":'int'})                   
student_bins=[0,1,11,51,50000]                        #범주화  0:인원미정/1-10(명):소/11-50(명):중/51-50000(명):대
student_scale=["인원미정","소","중","대"]               
df["선발인원규모"]=pd.cut(df["선발 인원(명)"],bins=student_bins,right=False,labels=student_scale) #분포범주화

#장학혜택
df["장학혜택"]=df["장학혜택"].str.replace(',',"")           #쉼표없애기(금액 때문)
df["장학혜택(만원)"]=df["장학혜택"].str.extract(r'(\d+)')   #숫자만남기기
df=df.replace({"장학혜택(만원)":np.NaN},0)                  #NaN(미정)값 0으로 변경                   
df=df.astype({"장학혜택(만원)":'int'})
money_bins=[0,1,101,201,50000]                               #범주화   0:금액미정/1-100(만원):소/101-200(만원):중/201-50000(만원):대
money_scale=["금액미정","소","중","대"]
df["장학혜택규모"]=pd.cut(df["장학혜택(만원)"],bins=money_bins,right=False,labels=money_scale)  #분포범주화

#열제거
df=df.drop(["장학금명","신청기간","조회수","선발인원","장학혜택"],axis='columns')

#열위치조정
df = df.reindex(columns=["인덱스","재단","장학종류","근로장학","기숙사/주거지원","기타장학","대출지원","연수/연구장학","장학금","취업연계/산학장학","선발대상","고등학생","기타/일반","대학생","대학원생","전문대생","중학생","청년","초등학생","선발 인원(명)","선발인원규모","장학혜택(만원)","장학혜택규모","조회 수"])

#csv파일저장
df=df.to_csv('scholarship.csv',encoding = 'euc-kr')

In [19]:
df.dtypes

인덱스             int64
재단             object
장학종류           object
근로장학            int64
기숙사/주거지원        int64
기타장학            int64
대출지원            int64
연수/연구장학         int64
장학금             int64
취업연계/산학장학       int64
선발대상           object
고등학생            int64
기타/일반           int64
대학생             int64
대학원생            int64
전문대생            int64
중학생             int64
청년              int64
초등학생            int64
선발 인원(명)        int32
선발인원규모       category
장학혜택(만원)        int32
장학혜택규모       category
조회 수            int32
dtype: object

In [66]:
print("선발인원 기초통계량\n",df["선발 인원(명)"].describe())

선발인원 기초통계량
 count     101.000000
mean       86.326733
std       221.647157
min         0.000000
25%         0.000000
50%         7.000000
75%        90.000000
max      1448.000000
Name: 선발 인원(명), dtype: float64


In [55]:
df= df.replace(0, np.nan)
df.describe().dropna()
print('\n')
print("선발인원 기초통계량:\n",df["선발 인원(명)"].describe())



선발인원 기초통계량:
 count      58.000000
mean      150.327586
std       276.403048
min         2.000000
25%        11.000000
50%        53.000000
75%       136.750000
max      1448.000000
Name: 선발 인원(명), dtype: float64


In [62]:
print("장학혜택(만원) 기초통계량:\n",df["장학혜택(만원)"].describe())

장학혜택(만원) 기초통계량:
 count      64.000000
mean      430.828125
std       777.975957
min        10.000000
25%       100.000000
50%       200.000000
75%       400.000000
max      5000.000000
Name: 장학혜택(만원), dtype: float64


In [57]:
df= df.replace(0, np.nan)
df.describe().dropna()
print('\n')
print("선발인원 기초통계량:\n",df["선발 인원(명)"])



선발인원 기초통계량:
 0       14.0
1        NaN
2        NaN
3      530.0
4        NaN
       ...  
96     280.0
97     149.0
98     100.0
99       NaN
100      NaN
Name: 선발 인원(명), Length: 101, dtype: float64


In [60]:
df= df.replace(0, np.nan)
df.describe().dropna()
df.loc[:, '선발 인원(명)']

0       14.0
1        NaN
2        NaN
3      530.0
4        NaN
       ...  
96     280.0
97     149.0
98     100.0
99       NaN
100      NaN
Name: 선발 인원(명), Length: 101, dtype: float64